# OpenAI Java SDK Overview

## 1. Introduction and Installation

### What is OpenAI Java SDK?

[OpenAI Java SDK](https://github.com/openai/openai-java/tree/main) is an official Java library generated by
[Stainless](https://www.stainless.com/) in pure Kotlin for interacting with the OpenAI API.
It allows developers to easily integrate language model capabilities and other OpenAI AI services into their JVM applications.


### Installation and Environment Setup

Using OpenAI Java SDK is straightforward.

For JVM projects:

```kotlin
implementation("com.openai:openai-java:$sdkVersion)
```

In Kotlin Notebook:

In [1]:
@file:DependsOn("com.openai:openai-java:0.41.1")

#### API Key

To work with the API, you need an API key that can be obtained from your personal account on the OpenAI website.

Add it to your environment variables.

[MacOS/Linux]
```bash
export OPENAI_API_KEY=<INSERT KEY HERE>
```

[Windows]
```shell
set OPENAI_API_KEY=<INSERT KEY HERE>
```

In [2]:
import com.openai.client.okhttp.OpenAIOkHttpClient

// Create a client using your API key from environment variables
// Configures using the `OPENAI_API_KEY`, `OPENAI_ORG_ID` and `OPENAI_PROJECT_ID` environment variables
val client = OpenAIOkHttpClient.fromEnv()

Let's create a simple request.

In [3]:
import com.openai.models.ChatModel
import com.openai.models.chat.completions.ChatCompletionCreateParams

val response = client.chat().completions().create(
    ChatCompletionCreateParams.builder()
        .model(ChatModel.GPT_4O_MINI)
        .addSystemMessage("You are a helpful assistant.")
        .addUserMessage("Hello! Who are you?")
        .build()
)

response.choices().first().message().content().get()

Hello! I’m an AI assistant designed to help you with information, answer questions, and assist with a variety of topics. How can I help you today?

## 2. Basic

#### Text Requests

Basic example of interacting with the model

In [4]:
val response = client.chat().completions().create(
    ChatCompletionCreateParams.builder()
        .model(ChatModel.GPT_4O_MINI)
        .addSystemMessage("You are a Kotlin expert. You can answer questions about Kotlin.")
        .addUserMessage("Explain what coroutines are in Kotlin and provide an example.")
        .build()
)

response.choices().first().message().content().get()

Coroutines in Kotlin are a powerful feature that simplifies asynchronous programming and concurrency. They allow you to write non-blocking code in a sequential style, making it easier to work with asynchronous tasks without the complexity of traditional callback methods or thread management.

Coroutines are lightweight threads that can be suspended and resumed, enabling you to perform long-running tasks without blocking the main thread. They are particularly useful for tasks like network calls, file I/O, or any operation that might take time and would otherwise cause your application to become unresponsive if handled on the main thread.

### Key Concepts of Coroutines
- **Suspend Functions**: Functions marked with the `suspend` keyword can be paused and resumed later. They can call other suspend functions, but they need to be called from within a coroutine or another suspend function.
- **Coroutine Builders**: Functions that help create coroutines, such as `launch`, `async`, and `runBl

### Configuring Parameters

OpenAI SDK allows configuring various parameters that affect text generation:

In [5]:
val response = client.chat().completions().create(
    ChatCompletionCreateParams.builder()
        .model(ChatModel.GPT_4O_MINI)
        .addSystemMessage("You are a creative writer.")
        .addUserMessage("Come up with a short story about a space voyage.")
        .temperature(0.8)           // Controls randomness (0-2, default is 1)
        .maxCompletionTokens(300)   // Maximum response length
        .topP(0.95)                 // Nucleus sampling - alternative to temperature
        .presencePenalty(0.2)       // Penalty for topic repetition (-2.0 to 2.0)
        .frequencyPenalty(0.5)      // Penalty for phrase repetition (-2.0 to 2.0)
        .build()
)

response.choices().first().message().content().get()

**Title: The Last Signal**

In the year 2267, humanity had expanded far beyond the cradle of Earth, establishing colonies on distant planets and moons. Among these ventures was the starship *Odyssey*, a sleek vessel tasked with exploring the uncharted reaches of the Andromeda sector. Captain Elara Voss stood on the bridge, her eyes fixed on the swirling cosmos outside.

“Prepare for jump in thirty seconds,” she commanded, her voice steady despite the excitement that buzzed through her crew. They were about to embark on a voyage that would take them deeper into space than any human had ever ventured before.

As they transitioned through hyperspace, time seemed to stretch and warp around them. For a brief moment, Elara found herself lost in thought about what lay ahead: new worlds, new lifeforms—perhaps even ancient civilizations hidden among the stars.

The ship jolted violently as they exited hyperspace. Alarms blared, and lights flickered ominously. “Status report!” Elara shouted as s

### Prompt Examples

The quality of the response heavily depends on the request structure

In [6]:
// Example 1: Basic prompt
val basicPrompt = "Explain machine learning."

// Example 2: Improved prompt
val goodPrompt =
    """
    Explain the concept of machine learning to a beginner student.
    Include the following:
    1) a definition in simple words
    2) the difference between supervised and unsupervised learning
    3) three real-world application examples
    Use analogies to explain complex concepts and keep the explanation within 200 words.
    """.trimIndent()

// Сравним результаты
for (prompt in listOf(basicPrompt, goodPrompt)) {
    val response = client.chat().completions().create(
        ChatCompletionCreateParams.builder()
            .model(ChatModel.GPT_4O_MINI)
            .addSystemMessage("You are a scientific expert.")
            .addUserMessage(prompt)
            .maxCompletionTokens(150)
            .build()
    )

    println("Prompt: $prompt")
    println("Response:\n${response.choices().first().message().content().get()}")
    println("-".repeat(80))
}

Prompt: Explain machine learning.
Response:
Machine learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks without being explicitly programmed. It allows systems to learn from data, improve their performance over time, and make predictions or decisions based on patterns and insights derived from that data.

### Key Concepts of Machine Learning:

1. **Data**: The foundation of machine learning is data. This can be structured (like tables) or unstructured (like text, images, or audio). Quality and quantity of data significantly impact the performance of ML models.

2. **Algorithms**: Machine learning uses a variety of algorithms to analyze data and identify patterns. Common types include:
   - **Supervised Learning**
--------------------------------------------------------------------------------
Prompt: Explain the concept of machine learning to a beginner student.
Inc

## 3. Function Calling

Function calling allows models to call specific functions,
which is especially useful for extracting structured data from text or
performing specific actions based on user input.

In [7]:
import com.openai.core.JsonValue
import com.openai.models.FunctionDefinition
import com.openai.models.FunctionParameters
import com.openai.models.chat.completions.ChatCompletionTool

fun getWeather(location: String): Map<String, Any> = when {
    "Berlin" in location -> mapOf("temp" to 13, "conditions" to "Cloudy")
    "Tokyo" in location -> mapOf("temp" to 21, "conditions" to "Sunny")
    "New York" in location -> mapOf("temp" to 15, "conditions" to "Rainy")
    else -> mapOf("temp" to 0, "conditions" to "Unknown")
}

val functionDef = FunctionDefinition.builder()
    .name("get_weather")
    .description("Get the weather conditions in a given location")
    .parameters(
        FunctionParameters.builder()
            .putAdditionalProperty("type", JsonValue.from("object"))
            .putAdditionalProperty(
                "properties", JsonValue.from(
                    mapOf(
                        "location" to JsonValue.from(
                            mapOf(
                                "type" to "string",
                                "description" to "The city or location to get the weather for"
                            )
                        )
                    )
                )
            )
            .putAdditionalProperty("required", JsonValue.from(listOf("location")))
            .putAdditionalProperty("additionalProperties", JsonValue.from(false))
            .build()
    )
    .build()

val userQuery = "What is the weather in Berlin?"

val params = ChatCompletionCreateParams.builder()
    .model(ChatModel.GPT_4O_MINI)
    .addUserMessage(userQuery)
    .addTool(ChatCompletionTool.builder().function(functionDef).build())


val response = client.chat().completions().create(params.build())

val aiMessage = response.choices().first().message()

"Response: $aiMessage"

Response: ChatCompletionMessage{content=null, refusal=null, role=assistant, annotations=[], audio=, functionCall=, toolCalls=[ChatCompletionMessageToolCall{id=call_OjCBz1vP0J7BdY2ntTF8yILW, function=Function{arguments={"location":"Berlin"}, name=get_weather, additionalProperties={}}, type=function, additionalProperties={}}], additionalProperties={}}

In [8]:
import com.openai.core.jsonMapper
import com.openai.models.chat.completions.ChatCompletionToolMessageParam
import kotlin.jvm.optionals.getOrNull

if (aiMessage.toolCalls().getOrNull()?.isNotEmpty() ?: false) {
    val functionCall = aiMessage.toolCalls().get().first()
    val functionName = functionCall.function().name()
    val functionArgs = jsonMapper().readValue(functionCall.function().arguments(), Map::class.java)

    println("The model wants to call a function: $functionName")
    println("With arguments: $functionArgs")

    if (functionName == "get_weather") {
        val location = functionArgs["location"] as? String ?: "Unknown"
        val funResponse = getWeather(location)

        val finalResponse = client.chat().completions().create(
            params
                .addMessage(aiMessage)
                .addMessage(
                    ChatCompletionToolMessageParam.builder()
                        .toolCallId(functionCall.id())
                        .content(funResponse.values.toString())
                        .build()
                )
                .build()
        )

        println("Final answer:")
        println(finalResponse.choices().first().message().content().getOrNull())
    }
}

The model wants to call a function: get_weather
With arguments: {location=Berlin}
Final answer:
The weather in Berlin is currently 13°C and cloudy.
